In [16]:
import random
from enum import StrEnum
from threading import ExceptHookArgs
from typing import Self

from rich.console import Console
from rich.panel import Panel
from rich.table import Table

console = Console()

def prompt_panel(text, color="bold cyan"):
    console.print(Panel(text, style=color))


def print_armies_summary(army_list):
    table = Table(title="[bold blue]Сводка армий[/]", style="cyan")
    table.add_column("Армия", style="magenta")
    table.add_column("Кол-во живых", style="green")
    table.add_column("HP живых жуков", style="yellow")
    for army in army_list:
        alive = [b for b in army.beetles_list if b.health_points > 0]
        names_hp = ", ".join(f"{b.health_points}" for b in alive)
        table.add_row(str(army.beetles_name), str(len(alive)), names_hp)
    console.print(table)

def victory_msg(army):
    console.print(Panel(f"[bold green]Армия {army.beetles_name} победила![/]", style="bold green"))

class Names(StrEnum):
    JOHN = "John Lennon"
    PAUL = "Paul McCartney"
    GEORGE = "George Harrison"
    RINGO = "Ringo Starr"


class Beetle:
    health_points: int
    name: Names
    damage: int

    def __init__(
        self,
        health_points: int = 100,
        damage: int = 10,
        name: Names = Names.JOHN,
    ) -> None:
        self.health_points = health_points
        self.name = name
        self.damage = damage

    def __eq__(self, other: Self) -> bool:
        return self.health_points == other.health_points

    def __lt__(self, other: Self) -> bool:
        return self.health_points < other.health_points

    def __le__(self, other: Self) -> bool:
        return self.health_points <= other.health_points

    def __str__(self) -> str:
        return f'Beetle(name="{self.name}", hp={self.health_points!r})'

    def styling(self) -> str:
        if self.name is Names.JOHN:
            return "in Johny style"
        elif self.name is Names.PAUL:
            return "in McCartney style"
        return "without style"

    def attack(self, other: Self) -> None:
        pastHP = other.health_points
        if other.health_points - self.damage <= 0:
            other.health_points = 0
        else:
            other.health_points -= self.damage
        console.print(
            f"[bold magenta]{self.name}[/] атакует [bold yellow]{other.name}[/] "
            f"[italic]{self.styling()}[/] | {pastHP}HP → [red]{other.health_points}HP[/]"
        )   



class BeetlesArmy:
    beetles_list: list[Beetle]
    beetles_name: Names
    beetles_max_health_points: int
    beetles_max_damage: int
    def __init__(
        self,
        beetles_name: Names,
        beetles_army_size: int = 20,
        beetles_max_health_points: int = 100,
        beetles_max_damage: int = 10,
    ):
        self.beetles_list = []
        self.beetles_name = beetles_name
        self.beetles_max_health_points = beetles_max_health_points
        self.beetles_max_damage = beetles_max_damage

        for _ in range(beetles_army_size):
            beetle = Beetle(
                health_points=random.randint(1, self.beetles_max_health_points),
                name=self.beetles_name,
                damage=random.randint(1, self.beetles_max_damage),
            )
            self.beetles_list.append(beetle)

    def __len__(self) -> int:
        return len(self.beetles_list)

    def __add__(self, other: Self) -> Self:
        if self.beetles_name != other.beetles_name:
            raise ValueError("Cannot make two different-named beetles friends")
        new_beetles_list: list[Beetle] = self.beetles_list + other.beetles_list
        new_army = self.__class__(
            beetles_army_size=1,
            beetles_name=self.beetles_name,
            beetles_max_health_points=self.beetles_max_health_points,
            beetles_max_damage=self.beetles_max_damage,
        )
        new_army.beetles_list = new_beetles_list
        return new_army

    def battle_with(self, other: Self) -> None:
        if not other.beetles_list:
            return
        for beetle in self.beetles_list:
            if len(other.beetles_list) == 0:
                break
            other_beetle = random.choice(other.beetles_list)
            beetle.attack(other_beetle)
            if other_beetle.health_points <= 0:
                pastHP = beetle.health_points
                if beetle.health_points + 10 > self.beetles_max_health_points:
                    beetle.health_points = self.beetles_max_health_points
                else:
                    beetle.health_points += 10
                console.print(
                    Panel(
                        f"[red]{other_beetle.name} погиб![/] "
                        f"[green]{beetle.name}[/] лечится: {pastHP}HP → [yellow]{beetle.health_points}HP[/]",
                        style="bright_red"
                    )
                )
                other.beetles_list.remove(other_beetle)
    
    def is_dead(self):
        for beetle in self.beetles_list:
            if beetle.health_points > 0:
                return False
        else:
            return True
    

    def rich_army_listing(self):
        table = Table(title=str(self.beetles_name), style="magenta")
        table.add_column("[bold green]#")
        table.add_column("Имя жука", style="bold blue")
        table.add_column("HP", style="bold yellow")
        table.add_column("Урон", style="bold red")
        for idx, beetle in enumerate(self.beetles_list, 1):
            table.add_row(str(idx), str(beetle.name), str(beetle.health_points), str(beetle.damage))
        console.print(table)

    


if __name__ == "__main__":

    armyList : list[BeetlesArmy] = []
    prompt_panel("Укажи кол-во армий:")
    for _ in range(int(input())):
        prompt_panel("Укажи имя армии (PAUL/JOHN):")
        name = Names[input()]
        prompt_panel("Укажи размер армии:")
        size = int(input())
        prompt_panel("Укажи максимальное здоровье жука:")
        hp = int(input())
        prompt_panel("Укажи максимальный урон жука:")
        damage = int(input())
        armyList.append(BeetlesArmy(name, size, hp, damage))
        print()
        armyList[-1].rich_army_listing()
        print()

    def end_game(armyList:list[BeetlesArmy]):
        for army in armyList:
            if army.is_dead():
                console.print(f"[bold]Армия {army.beetles_name} проиграла[/]")
                armyList.remove(army)
        if len(armyList) == 1:
            victory_msg(armyList[0])
            return True
        return False
    
    
 
    while not end_game(armyList):
        print_armies_summary(armyList)
        for army in armyList:
            number = random.randint(0, len(armyList) - 1)
            while number == armyList.index(army):
                number = random.randint(0, len(armyList) - 1)
            army.battle_with(armyList[number])

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Укажи кол-во армий:                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Укажи имя армии (PAUL/JOHN):                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Укажи размер армии:                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Укажи максимальное здоровье жука:                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Укажи максимальный урон жука:                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ John Lennon погиб! Paul McCartney лечится: 1HP → 3HP                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Paul McCartney атакует John Lennon in McCartney style | 1HP → 0HP

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ John Lennon погиб! Paul McCartney лечится: 2HP → 3HP                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Армия John Lennon проиграла

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Армия Paul McCartney победила!                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯